In [1]:
import pandas as pd

In [2]:
payload = pd.read_pickle('data/data_file.pkl')
hand_df = payload["hand_data"].copy()
eit_df = payload["eit_data"].copy()

In [3]:
print(eit_df.head())
print(eit_df["Event"].unique())
print(eit_df.isna().sum())
print(eit_df.count())
len(eit_df)

             timestamp Event            Phase  Frame_ID  Inj  Sense     Real  \
0  1772145457803495500        Pre-Calibration     354.0  0.0    4.0 -14943.0   
1  1772145457803495500        Pre-Calibration     354.0  1.0    0.0    756.0   
2  1772145457803495500        Pre-Calibration     354.0  1.0    1.0   1064.0   
3  1772145457804980800        Pre-Calibration     354.0  1.0    2.0    861.0   
4  1772145457805996900        Pre-Calibration     354.0  1.0    3.0    865.0   

       Img     Magnitude  
0 -42771.0  45306.199245  
1 -13310.0  13331.452884  
2 -12915.0  12958.754608  
3 -13238.0  13265.970187  
4 -12869.0  12898.038068  
['' 'baseline_started' 'baseline_done']
timestamp    0
Event        0
Phase        2
Frame_ID     2
Inj          2
Sense        2
Real         2
Img          2
Magnitude    2
dtype: int64
timestamp    78158
Event        78158
Phase        78156
Frame_ID     78156
Inj          78156
Sense        78156
Real         78156
Img          78156
Magnitude    7815

78158

In [4]:
col = "Event" 

i = eit_df.index[eit_df[col].eq("baseline_done")].min() 
eit_df = eit_df.loc[i:].iloc[1:] if pd.notna(i) else eit_df 

In [5]:
print(eit_df.count())
len(eit_df)
eit_df = eit_df.drop(columns=["Event"])

timestamp    75560
Event        75560
Phase        75560
Frame_ID     75560
Inj          75560
Sense        75560
Real         75560
Img          75560
Magnitude    75560
dtype: int64


In [6]:
eit_df["timestamp"] = eit_df["timestamp"].astype("int64")
print(eit_df["timestamp"].unique())

[1772145468047259200 1772145468049349800 1772145468051358300 ...
 1772145765701916000 1772145765702924400 1772145765704267000]


In [7]:
print(eit_df.head(50))

                timestamp        Phase  Frame_ID  Inj  Sense         Real  \
2598  1772145468047259200  Measurement     419.0  0.0    0.0   399.526316   
2599  1772145468047259200  Measurement     419.0  0.0    1.0  -101.157895   
2600  1772145468047259200  Measurement     419.0  0.0    2.0  -117.578947   
2601  1772145468047259200  Measurement     419.0  0.0    3.0 -1294.631579   
2602  1772145468047259200  Measurement     419.0  0.0    4.0  -648.315789   
2603  1772145468049349800  Measurement     419.0  1.0    0.0   115.052632   
2604  1772145468049349800  Measurement     419.0  1.0    1.0   -28.105263   
2605  1772145468049349800  Measurement     419.0  1.0    2.0  -200.368421   
2606  1772145468049349800  Measurement     419.0  1.0    3.0  -252.842105   
2607  1772145468051358300  Measurement     419.0  1.0    4.0   -92.526316   
2608  1772145468051358300  Measurement     419.0  2.0    0.0  1057.210526   
2609  1772145468051358300  Measurement     419.0  2.0    1.0  1258.789474   

In [ ]:
import numpy as np
import pandas as pd

TS  = "timestamp"
FID = "Frame_ID"
R   = "Inj"
C   = "Sense"

V = ["Real", "Img", "Magnitude"]

df = eit_df.copy()
df[R] = df[R].astype(int)
df[C] = df[C].astype(int)

# record index 0..39 (8*5=40) based on row/col
df["rec"] = df[R] * 5 + df[C]

# average timestamp per frame (common definition: midpoint)
avg_ts = df.groupby(FID)[TS].agg(lambda s: (s.min() + s.max()) / 2).astype("int64")

# build a complete frame x rec index so every frame has exactly 40 records
full_index = pd.MultiIndex.from_product([avg_ts.index, range(40)], names=[FID, "rec"])

# keep only what we need, align to full grid, fill missing
wide = (
    df.set_index([FID, "rec"])[V]
      .reindex(full_index)
      .fillna(0.0)
      .sort_index()
)

# tensor (num_frames, 40, 3)
X = wide.to_numpy().reshape(len(avg_ts), 40, len(V))

# timestamps (num_frames,)
T = avg_ts.to_numpy()

print("X:", X.shape, "T:", T.shape)

T3 = np.repeat(T[:, None, None], repeats=len(V), axis=2)  # (N,1,3)
X_with_time = np.concatenate([T3, X], axis=1)             # (N,41,3)
print(X_with_time.shape)

Frame_ID
419.0    1772145468057779712
420.0    1772145468214769920
421.0    1772145468369596928
422.0    1772145468524510720
423.0    1772145468687997952
Name: timestamp, dtype: int64
X: (1889, 40, 3) T: (1889,)
(1889, 41, 3)


In [9]:
print(X_with_time[0, 0])  # timestamp of first frame
print(X_with_time[0, :]) # 40 records of first frame

[1.77214547e+18 1.77214547e+18 1.77214547e+18]
[[ 1.77214547e+18  1.77214547e+18  1.77214547e+18]
 [ 3.99526316e+02 -9.33473684e+02  9.30475822e+02]
 [-1.01157895e+02  1.16678947e+03 -1.16603591e+03]
 [-1.17578947e+02 -8.26684211e+02  8.27240055e+02]
 [-1.29463158e+03 -1.75842105e+03  1.79981531e+03]
 [-6.48315789e+02 -2.00252632e+03  2.00806935e+03]
 [ 1.15052632e+02 -2.67894737e+01  3.45140951e+01]
 [-2.81052632e+01  1.05789474e+01 -1.20875644e+01]
 [-2.00368421e+02  1.09263158e+02 -1.18479545e+02]
 [-2.52842105e+02 -2.19000000e+02  2.04723659e+02]
 [-9.25263158e+01  4.42631579e+01 -4.57764482e+01]
 [ 1.05721053e+03  1.00368421e+02 -9.82384293e+01]
 [ 1.25878947e+03 -1.55315789e+03  1.57541419e+03]
 [ 1.19215789e+03 -1.11789474e+02  1.11732375e+02]
 [ 1.26010526e+03  9.03157895e+01 -7.39093080e+01]
 [ 5.47421053e+02 -1.78710526e+03  1.77574324e+03]
 [ 1.96157895e+02  5.94736842e+01 -4.83653375e+01]
 [-3.07894737e+01 -1.52105263e+02  1.50418301e+02]
 [ 9.42631579e+01 -2.72105263e+01  